# 基于形状的模板匹配
使用了Opencv自带的matchShapes()方法

### 辅助类

In [2]:
import cv2
class MyFrame:
    def __init__(self,init_frame):
        self.frame = init_frame
    def grayscale(self,color=cv2.COLOR_BGR2GRAY):
        self.frame = cv2.cvtColor(self.frame,color)
        return self
    def gaussian_blur(self):
        self.frame = cv2.GaussianBlur(self.frame,(5, 5), 1, 0)
        return self
    def canny(self):
        self.frame = cv2.Canny(self.frame,128,200)
        return self
    def threshold(self,threshold_value=127,max_val=255,method=cv2.THRESH_BINARY):
        ret, frame = cv2.threshold(self.frame, threshold_value, max_val, method)
        self.frame = frame
        return self
    def dilate(self,kernel_size=(3,3),iters=1):
        kernel = np.ones(kernel_size, dtype=np.uint8)
        self.frame = cv2.dilate(self.frame, kernel, iters)
        return self

## 检测三角形

In [5]:
def detect_triangle(frame,template_frame):
    template = load_template('template_6.jpg')
    is_triangle_found = False
    
    low=20
    high=30
    hsv=cv2.cvtColor(frame,cv2.COLOR_RGB2HSV)
    # lower_yellow=np.array([low,50,50])#yellow
    # upper_yellow=np.array([high,255,255])#yellow
    lower_yellow=np.array([100,20,180])#red
    upper_yellow=np.array([255,255,238])#red
    mask = cv2.inRange(hsv,lower_yellow,upper_yellow)
    mb = mask.copy()
    # mask = cv2.cvtColor(mask,cv2.COLOR_HSV2RGB)
    myframe = MyFrame(mask)
    #myframe.gaussian_blur().threshold().canny().dilate(iters=15)
    myframe.frame =cv2.adaptiveThreshold(myframe.frame,210,cv2.BORDER_REPLICATE,cv2.THRESH_BINARY_INV,3,10)
    myframe.dilate(iters=15)
    contours, hierarchy = cv2.findContours(myframe.frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        draw_frame = myframe.frame
        # cv2.drawContours(frame, [approx], 0, (0,255,0), 3)
        # cv2.drawContours(draw_frame, [approx], 0, (0,255,0), 3)
        
    min_pos = -1
    min_value = 2
    percent = 0
    testing = True
    for i in range(len(contours)):
        # 参数3：匹配方法；参数4：opencv预留参数
        value = cv2.matchShapes(template,contours[i],1,0.0)
        if value < min_value and cv2.contourArea(contours[i]) > 5000:
            min_value = value
            min_pos = i
    # 参数3为0表示绘制本条轮廓contours[min_pos]
    
    if min_pos >=0 and min_pos < len(contours):
        triangle_contour = contours[min_pos]
    else:
        triangle_contour = None
    return myframe,triangle_contour,min_value


In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
template_frame = cv2.imread('template_6.jpg',cv2.CV_8UC1)
template_frame = cv2.resize(template_frame,(256,256))
template_dst = cv2.adaptiveThreshold(template_frame,210,cv2.BORDER_REPLICATE,cv2.THRESH_BINARY_INV,3,10)
template_contours,template_heridency = cv2.findContours(template_dst,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
template = template_contours[0]
print(len(template_contours))
run = True
while run:
    ret,frame = cap.read()
    if frame is None:
        continue
    
    frame = cv2.resize(frame,(512,512))
    # match_frame = cv2.resize(match_frame,(512,512))

    # cv2.drawContours(template_frame, template, -1, (255,0,255), 10)
    
    low=20
    high=30
    hsv=cv2.cvtColor(frame,cv2.COLOR_RGB2HSV)
    # lower_yellow=np.array([low,50,50])#yellow
    # upper_yellow=np.array([high,255,255])#yellow
    lower_yellow=np.array([100,20,180])#red
    upper_yellow=np.array([255,255,238])#red
    mask = cv2.inRange(hsv,lower_yellow,upper_yellow)
    mb = mask.copy()
    # mask = cv2.cvtColor(mask,cv2.COLOR_HSV2RGB)
    myframe = MyFrame(mask)
    #myframe.gaussian_blur().threshold().canny().dilate(iters=15)
    myframe.frame =cv2.adaptiveThreshold(myframe.frame,210,cv2.BORDER_REPLICATE,cv2.THRESH_BINARY_INV,3,10)
    myframe.dilate(iters=15)
    contours, hierarchy = cv2.findContours(myframe.frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
    min_pos = -1
    min_value = 2
    percent = 0
    testing = True
    for i in range(len(contours)):
        # 参数3：匹配方法；参数4：opencv预留参数
        value = cv2.matchShapes(template,contours[i],1,0.0)
        if value < min_value and cv2.contourArea(contours[i]) > 5000:
            min_value = value
            min_pos = i
    # 参数3为0表示绘制本条轮廓contours[min_pos]
    if min_pos >=0:
        percent = (1-min_value)*100
        if percent >= 80 or testing==True:
            cv2.drawContours(frame,[contours[min_pos]],0,[255,0,0],3)

    frame = cv2.putText(frame, 'match rate:'+str(min_value), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.3, (255,255,0), 1, cv2.LINE_AA)
    frame = cv2.putText(frame, 'match similarity:'+str((1-min_value)*100)+"%", (50,150), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.3, (255,255,0), 1, cv2.LINE_AA)
    
    cv2.drawContours(frame,[contours[min_pos]],0,[255,0,0],3)

    cv2.imshow('capture',frame)
    cv2.imshow('processed',myframe.frame)
    cv2.imshow('template',template_dst)
    cv2.imshow('mb',mb)

    
    if cv2.waitKey(100) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

## 检测正方形

In [ ]:
def detect_rectangle(frame,template):
    low=20
    high=30
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    lower_yellow=np.array([0,0,0])
    upper_yellow=np.array([255,255,112])
    mask = cv2.inRange(hsv,lower_yellow,upper_yellow)
    # mask = cv2.cvtColor(mask,cv2.COLOR_HSV2RGB)
    myframe = MyFrame(mask)
    myframe.gaussian_blur().threshold().canny().dilate(iters=100)
    contours, hierarchy = cv2.findContours(myframe.frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        draw_frame = myframe.frame
        # cv2.drawContours(frame, [approx], 0, (0,255,0), 3)
        # cv2.drawContours(draw_frame, [approx], 0, (0,255,0), 3)
        
    min_pos = -1
    min_value = 2
    for i in range(len(contours)):
        # 参数3：匹配方法；参数4：opencv预留参数
        value = cv2.matchShapes(template,contours[i],1,0.0)
        if value < min_value and cv2.contourArea(contours[i]) > 1000:
            min_value = value
            min_pos = i
            
    # 参数3为0表示绘制本条轮廓contours[min_pos]
    if min_pos >=0:
        rect_contour = contours[min_pos]
        # cv2.drawContours(frame,[contours[min_pos]],0,[255,0,0],3)
        # match_shape = contours[min_pos]
        # rect=cv2.minAreaRect(match_shape)
        # angle = rect[2]
        # frame = cv2.putText(frame, 'angle:'+str(angle), (50,110), cv2.FONT_HERSHEY_SIMPLEX, 
        #            0.5, (255,255,0), 1, cv2.LINE_AA)    # cv2.drawContours(frame,template,-1,[0,0,255],10)
    else:
        rect_contour = None
    return myframe,rect_contour,min_value

In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
template_frame = cv2.imread('template_4.jpg',cv2.CV_8UC1)
template_frame = cv2.resize(template_frame,(512,512))
template_dst = cv2.adaptiveThreshold(template_frame,210,cv2.BORDER_REPLICATE,cv2.THRESH_BINARY_INV,3,10)
template_contours,template_heridency = cv2.findContours(template_dst,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
template = template_contours[0]

print(len(template_contours))
run = True
while run:
    ret,frame = cap.read()
    if frame is None:
        continue
    
    frame = cv2.resize(frame,(512,512))

    myframe,rect_contour,match_rate = detect_rectangle(frame,template)
    if rect_contour is not None:
        cv2.drawContours(frame,[rect_contour],0,[255,0,0],3)


    cv2.imshow('capture',frame)
    cv2.imshow('capture2',myframe.frame)

    
    if cv2.waitKey(100) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

## 先检测出三角形，然后检测正方形

In [ ]:
def load_template(filename):
    template_frame = cv2.imread(filename,cv2.CV_8UC1)
    template_frame = cv2.resize(template_frame,(256,256))
    template_dst = cv2.adaptiveThreshold(template_frame,210,cv2.BORDER_REPLICATE,cv2.THRESH_BINARY_INV,3,10)
    template_contours,template_heridency = cv2.findContours(template_dst,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    template = template_contours[0]
    return template

def cv2ImgAddText(img, text, left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, numpy.ndarray)):  #判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontText = ImageFont.truetype(
        "font/Deng.ttf", textSize, encoding="utf-8")
    draw.text((left, top), text, textColor, font=fontText)
    return cv2.cvtColor(numpy.asarray(img), cv2.COLOR_RGB2BGR)

In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
template = load_template('template_6.jpg')
rect_template = load_template('template_4.jpg')
# print(len(template_contours))
run = True
while run:
    ret,frame = cap.read()
    if frame is None:
        continue
    
    frame = cv2.resize(frame,(512,512))

    myframe,triangle_contour,match_rate = detect_triangle(frame,template)
    if triangle_contour is not None:
        cv2.drawContours(frame,[triangle_contour],0,[255,0,0],3)
        rectframe,rect_contour,match_rate= detect_rectangle(frame,rect_template)
        if rect_contour is not None:
            cv2.drawContours(frame,[rect_contour],0,[255,255,0],3)

    frame = cv2.putText(frame, 'match rate:'+str(match_rate), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.4, (255,255,0), 1, cv2.LINE_AA)
    frame = cv2.putText(frame, 'match similarity:'+str((1-match_rate)*100)+"%", (50,130), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.4, (255,255,0), 1, cv2.LINE_AA)


    cv2.imshow('capture',frame)
    cv2.imshow('capture2',myframe.frame)
    
    if cv2.waitKey(100) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()